## *Analyzing StatsBomb Shot Freeze Frames and Constructing an xG Model*
    - Retrieve details regarding shots and their attributes.
    - Investigate the use and information extracted from shot freeze frames.
    - Extract new features.
    - Develop a predictive model for expected goals.
    - Evaluate the performance and accuracy of the created model.

In [10]:
import os
import json
import pandas as pd
import math 
import matplotlib.pyplot as plt 
from pandas.io.json import json_normalize

In [25]:
match_events_path= 'data/events/7451.json'
with open(match_events_path, 'r') as file:
    data = json.load(file)
df = pd.json_normalize(data)

shot_freeze_frame = df.loc[df['shot.freeze_frame'].notnull(), 'shot.freeze_frame'].iloc[0]
shot_freeze_frame

[{'location': [115.0, 42.0],
  'player': {'id': 5025, 'name': 'Aubrey Renee Kingsbury'},
  'position': {'id': 1, 'name': 'Goalkeeper'},
  'teammate': False},
 {'location': [112.0, 49.0],
  'player': {'id': 4962, 'name': 'Caprice Dydasco'},
  'position': {'id': 6, 'name': 'Left Back'},
  'teammate': False},
 {'location': [110.0, 36.0],
  'player': {'id': 5034, 'name': 'Jessica McDonald'},
  'position': {'id': 24, 'name': 'Left Center Forward'},
  'teammate': True},
 {'location': [112.0, 41.0],
  'player': {'id': 4950, 'name': 'Whitney Church'},
  'position': {'id': 5, 'name': 'Left Center Back'},
  'teammate': False},
 {'location': [113.0, 37.0],
  'player': {'id': 4951, 'name': 'Quinn Quinn'},
  'position': {'id': 3, 'name': 'Right Center Back'},
  'teammate': False}]

In [2]:
def process_json_file(file_path, shots_list):
    with open(file_path, 'r') as file:
        data = json.load(file)
        for item in data:
            shot = item.get('shot')
            location = item.get('location')
            if shot is not None:
                shots_list.append((shot, location))

In [3]:
def process_all_json_files_in_directory(directory):
    shots_list = []
    
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            process_json_file(file_path, shots_list)
    
    return shots_list

In [4]:
json_directory = 'data/events'
shots = process_all_json_files_in_directory(json_directory)
# Create a DataFrame out of proceesed data
df = pd.DataFrame(shots, columns=['Shot Value', 'Location'])
df

In [5]:
df

,Shot Value,Location
0,"{'statsbomb_xg': 0.06773658, 'end_location': [...","[107.5, 27.2]"
1,"{'statsbomb_xg': 0.017595131, 'end_location': ...","[88.3, 50.2]"
2,"{'statsbomb_xg': 0.7835, 'end_location': [120....","[108.0, 40.0]"
3,"{'one_on_one': True, 'statsbomb_xg': 0.4228571...","[107.3, 36.1]"
4,"{'statsbomb_xg': 0.077898495, 'end_location': ...","[105.3, 29.4]"
...,...,...
81112,"{'statsbomb_xg': 0.13645959, 'end_location': [...","[104.0, 36.1]"
81113,"{'statsbomb_xg': 0.2980615, 'end_location': [1...","[113.5, 43.0]"
81114,"{'statsbomb_xg': 0.04339284, 'end_location': [...","[103.5, 32.6]"
81115,"{'statsbomb_xg': 0.15713295, 'end_location': [...","[109.9, 29.9]"


In [11]:
freeze_frame= df['Shot Value'][0]['freeze_frame']
freeze_frame

[{'location': [100.2, 26.5],
  'player': {'id': 26198, 'name': 'Guillermo Luis Franco Farquarson'},
  'position': {'id': 23, 'name': 'Center Forward'},
  'teammate': True},
 {'location': [104.3, 40.7],
  'player': {'id': 19312, 'name': 'Robert Pirès'},
  'position': {'id': 19, 'name': 'Center Attacking Midfield'},
  'teammate': True},
 {'location': [106.5, 42.1],
  'player': {'id': 20177, 'name': 'Eric-Sylvain Bilal Abidal'},
  'position': {'id': 6, 'name': 'Left Back'},
  'teammate': False},
 {'location': [109.3, 30.5],
  'player': {'id': 25919, 'name': 'Gabriel Alejandro Milito'},
  'position': {'id': 5, 'name': 'Left Center Back'},
  'teammate': False},
 {'location': [116.1, 34.9],
  'player': {'id': 20176, 'name': 'Víctor Valdés Arribas'},
  'position': {'id': 1, 'name': 'Goalkeeper'},
  'teammate': False},
 {'location': [92.3, 35.2],
  'player': {'id': 19757, 'name': 'Anderson Luís de Souza'},
  'position': {'id': 15, 'name': 'Left Center Midfield'},
  'teammate': False},
 {'locat